In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from statsmodels.formula.api import ols
import statsmodels.api as sm
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.style.use('ggplot')
pd.options.mode.chained_assignment = None  # default='warn

In [96]:
df = pd.read_csv('total.csv')

In [97]:
df = df.reindex(columns = ['tr_log_testduration', 'tr_log_num_tests_failed', 'tr_log_lan', 'tr_log_num_tests_run','loc', 'tr_log_bool_tests_failed'])

In [98]:
df = df.rename(columns={'tr_log_testduration': 'time', 'tr_log_num_tests_failed': 'failed_count', 'tr_log_lan' : 'language', 'tr_log_num_tests_run' : 'test_count', 'tr_log_bool_tests_failed' : 'status'})

In [99]:
df = df[df['failed_count'].notnull() & df['test_count'].notnull() & df['status'].notnull()]

In [100]:
df.eval('log_test_count = log(test_count)', inplace = True)
df.eval('fail_rate = failed_count / test_count', inplace = True)

In [101]:
#df = df[(df['loc'] > 500) & (df['test_count'] > 1)]
df = df[(df['loc'] > 2000) &(df['loc'] < 200000) & (df['test_count'] > 1) & (df['time'] > 1)]

df = df[((df["loc"] - df["loc"].mean()).abs() < 3 * df["loc"].std()) & 
        ((df["log_test_count"] - df["log_test_count"].mean()).abs() < 3 * df["log_test_count"].std())]

In [102]:
df = df[(df['language'] == 'java') | (df['language'] == 'ruby') | (df['language'] == 'python')]

In [103]:
df['ruby'] = [1 if df.iloc[i, 2] == 'ruby' else 0 for i in range(len(df))]
df['python'] = [1 if df.iloc[i, 2] == 'python' else 0 for i in range(len(df))]
df['java'] = [1 if df.iloc[i, 2] == 'java' else 0 for i in range(len(df))]

In [104]:
df

,time,failed_count,language,test_count,loc,status,log_test_count,fail_rate,ruby,python,java
5,6.63,0.0,ruby,858.0,3969,False,6.754604,0.000000,1,0,0
6,4.48,0.0,ruby,916.0,4031,False,6.820016,0.000000,1,0,0
41,3.81,0.0,ruby,246.0,4583,False,5.505332,0.000000,1,0,0
42,3.87,108.0,ruby,246.0,4583,True,5.505332,0.439024,1,0,0
44,4.11,108.0,ruby,246.0,4581,True,5.505332,0.439024,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
31181,57.58,0.0,python,257.0,13793,False,5.549076,0.000000,0,1,0
31205,32.02,1.0,python,769.0,51334,True,6.645091,0.001300,0,1,0
31206,32.98,1.0,python,770.0,51245,True,6.646391,0.001299,0,1,0
31207,31.31,0.0,python,771.0,51267,False,6.647688,0.000000,0,1,0


In [107]:
y = (df['fail_rate'])
X = df.iloc[:,[4,6,8,10]]
#X = sm.add_constant(X)
model = sm.OLS(y, X)
results = model.fit()
results
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:              fail_rate   R-squared (uncentered):                   0.005
Model:                            OLS   Adj. R-squared (uncentered):              0.004
Method:                 Least Squares   F-statistic:                              8.637
Date:                Wed, 08 Dec 2021   Prob (F-statistic):                    5.97e-07
Time:                        17:20:37   Log-Likelihood:                          7599.3
No. Observations:                6987   AIC:                                 -1.519e+04
Df Residuals:                    6983   BIC:                                 -1.516e+04
Df Model:                           4                                                  
Covariance Type:            nonrobust                                                  
                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------